## Import

In [76]:
import numpy as np
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import fastai
from fastai import *
from fastai.data.core import DataLoaders
from fastai.learner import Learner
from fastai.losses import CrossEntropyLossFlat
from fastai.metrics import accuracy
from fastai.optimizer import Adam
from fastai.callback.progress import ProgressCallback

from parse_preprocessed_data import get_inputs_and_targets
from LSTM_Model import LSTMModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Hyper-Parameters

In [63]:
seq_length = 50

hidden_size = 128
learning_rate = 2e-3
dropout = 0.5
batch_size = 100
num_layers = 3
max_epochs = 20
validation_prop = 0.2

early_stopping = True
patience = 20

## Load Data

In [64]:
char_to_ix, ix_to_char, vocab_size, inputs, targets = get_inputs_and_targets('data_preprocessed/mario.txt', seq_length)
vocab_size, inputs.shape, targets.shape

Unique chars: ['\n', '-', '<', '>', '?', 'B', 'E', 'Q', 'S', 'X', '[', ']', 'b', 'o', 'x']
Number of unique chars: 15


  0%|          | 0/37 [00:00<?, ?it/s]

(15, (124700, 50, 15), (124700, 50))

In [65]:
first_three_cols = inputs[0][:3 * 17]
np.savetxt('data_preprocessed/seed.txt', first_three_cols)

In [66]:
with open('data_preprocessed/char_to_ix.json', 'w+') as json_f:
    json.dump(char_to_ix, json_f)

with open('data_preprocessed/ix_to_char.json', 'w+') as json_f:
    json.dump(ix_to_char, json_f)

In [67]:
inputs.shape,inputs.dtype

((124700, 50, 15), dtype('float64'))

In [68]:
targets.shape, targets.dtype

((124700, 50), dtype('int32'))

In [70]:
# Convert inputs and targets to PyTorch tensors
inputs_tensor = torch.tensor(inputs, dtype=torch.float32)
targets_tensor = torch.tensor(targets, dtype=torch.int32)

# Create Dataset
dataset = torch.utils.data.TensorDataset(inputs_tensor, targets_tensor)

# Create DataLoader
train_dl = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# For validation, ensure that you also provide inputs and targets as tensors and stack them into batches
valid_dl = DataLoader(dataset, batch_size=batch_size)

# Create DataLoaders object
dls = DataLoaders(train_dl, valid_dl)


## Model Callbacks

## Model

In [77]:
# Initialize model
model = LSTMModel(vocab_size=vocab_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)


In [78]:
learn = Learner(dls, model, opt_func=Adam, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

## Train Model

In [79]:
learn.fit(n_epoch=max_epochs, lr=learning_rate, cbs=ProgressCallback())

epoch,train_loss,valid_loss,accuracy,time


epoch,train_loss,valid_loss,accuracy,time
0,0.797792,0.818843,0.777133,00:08
1,0.778809,0.824250,0.774212,00:09
2,0.763178,0.819943,0.775746,00:08
3,0.759154,0.817647,0.774769,00:08
4,0.760720,0.795558,0.778740,00:08
5,0.758469,0.815887,0.777368,00:08
6,0.759833,0.813577,0.778058,00:08
7,0.756248,0.807253,0.776642,00:08
8,0.757285,0.800227,0.777876,00:09
9,0.760482,0.809132,0.780285,00:08


In [80]:
torch.save(model.state_dict(), 'trained_models/mario_lstm.h5')

In [81]:
learn.export('trained_learners/mario_lstm.pkl')

AttributeError: 'DataLoader' object has no attribute 'new'